# LangChain: Q&A over Documents

An example might be a tool that would allow you to query a product catalog for items of interest.

In [0]:
#pip install --upgrade langchain

In [0]:
import os

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

In [0]:
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import CSVLoader
from langchain.vectorstores import DocArrayInMemorySearch
from IPython.display import display, Markdown

In [0]:
file = 'OutdoorClothingCatalog_1000.csv'
loader = CSVLoader(file_path=file)

In [0]:
from langchain.indexes import VectorstoreIndexCreator

In [0]:
#pip install docarray

In [0]:
index = VectorstoreIndexCreator(
    vectorstore_cls=DocArrayInMemorySearch
).from_loaders([loader])

In [0]:
query ="Please list all your shirts with sun protection \
in a table in markdown and summarize each one."

In [0]:
response = index.query(query)

In [0]:
display(Markdown(response))

In [0]:
loader = CSVLoader(file_path=file)

In [0]:
docs = loader.load()

In [0]:
docs[0]

In [0]:
from langchain.embeddings import OpenAIEmbeddings
embeddings = OpenAIEmbeddings()

In [0]:
embed = embeddings.embed_query("Hi my name is Harrison")

In [0]:
print(len(embed))

In [0]:
print(embed[:5])

In [0]:
db = DocArrayInMemorySearch.from_documents(
    docs, 
    embeddings
)

In [0]:
query = "Please suggest a shirt with sunblocking"

In [0]:
docs = db.similarity_search(query)

In [0]:
len(docs)

In [0]:
docs[0]

In [0]:
retriever = db.as_retriever()

In [0]:
llm = ChatOpenAI(temperature = 0.0)


In [0]:
qdocs = "".join([docs[i].page_content for i in range(len(docs))])


In [0]:
response = llm.call_as_llm(f"{qdocs} Question: Please list all your \
shirts with sun protection in a table in markdown and summarize each one.") 


In [0]:
display(Markdown(response))

In [0]:
qa_stuff = RetrievalQA.from_chain_type(
    llm=llm, 
    chain_type="stuff", 
    retriever=retriever, 
    verbose=True
)

In [0]:
query =  "Please list all your shirts with sun protection in a table \
in markdown and summarize each one."

In [0]:
response = qa_stuff.run(query)

In [0]:
display(Markdown(response))

In [0]:
response = index.query(query, llm=llm)

In [0]:
index = VectorstoreIndexCreator(
    vectorstore_cls=DocArrayInMemorySearch,
    embedding=embeddings,
).from_loaders([loader])